In [8]:
# !pip install spacy

In [ ]:
!python -m spacy download en_core_web_sm

    17% |█████▌                          | 1.9MB 25kB/s eta 0:06:01

In [12]:
import os , re
import spacy
import numpy as np
import pandas as pd
import seaborn as sns

import gensim
import pyLDAvis.gensim

from sklearn.svm import LinearSVC
from sklearn.base import TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from sklearn.pipeline import Pipeline

# from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

nlp = spacy.load("en_core_web_sm")
SYMBOLS="?:!.,;'"
STOPLIST = list(ENGLISH_STOP_WORDS)
stemmer = SnowballStemmer("english", ignore_stopwords=True)

folders = './training/'

OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.

In [11]:
entities = os.listdir(folders)
print("Total: {} entities".format(len(entities)))

corpus = []
for value in entities:
    path = folders + value
    with open(path,'r') as f:
        data = f.read()
        corpus.append(data)
print("Total readable at: {} files".format(len(corpus)))


NameError: name 'folders' is not defined

In [ ]:
def cleanText(text):
    text = text.strip().replace("\n", " ").replace("\r", " ")
    text = text.lower()
    return text
def tokenizeText(sample):
    tokens = nlp(sample)
    lemmas = []
    for tok in tokens:
        lemmas.append(tok.lemma_.lower().strip() if tok.lemma_ != "-PRON-" else tok.lower_)
    tokens = lemmas
    tokens = [re.sub(r"([0-9]+)","",tok) for tok in tokens]
    tokens = [tok for tok in tokens if tok not in STOPLIST]
    tokens = [tok for tok in tokens if tok not in SYMBOLS]
    return tokens
# print(len(nlp(corpus[700]).text))
# test_token = tokenizeText(corpus[700])
# print(len(test_token))
# test_token

In [ ]:
class CleanTextTransformer(TransformerMixin):
    def transform(self, X, **transform_params):
        return [cleanText(text) for text in X]
    def fit(self, X, y=None, **fit_params):
        return self
    def get_params(self, deep=True):
        return {}

In [ ]:
vectorizer = CountVectorizer(tokenizer=tokenizeText, ngram_range=(1,1))
clf = LinearSVC()
pipe = Pipeline([('cleanText', CleanTextTransformer()), ('vectorizer', vectorizer)])
# pipe = Pipeline([('cleanText', CleanTextTransformer()), ('vectorizer', vectorizer), ('clf', clf)])

In [ ]:
train = corpus[:11000]
test = corpus[19001:21000]

In [ ]:
pipe.fit(train)

In [ ]:
processed_docs = pipe.transform(train)

In [ ]:
# Push to dictionary
dictionary = gensim.corpora.Dictionary(processed_docs)

In [ ]:
# "# I dont want words no_below, no_above
dictionary.filter_extremes(no_below=10, no_above=0.2,keep_n=1000)

In [ ]:
# "# Bag of words
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
# type(bow_corpus)

In [ ]:
# Fit LDA model\n",
# num_topic [dont know] sothat pick a proper number,
lda_model = gensim.models.LdaMulticore(bow_corpus,
                                       num_topics=7,
                                      id2word=dictionary,
                                     passes=10, workers=2)

In [ ]:
# LDA help vizualize
lda_display = pyLDAvis.gensim.prepare(lda_model, bow_corpus, dictionary)
pyLDAvis.display(lda_display)